In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer, BertTokenizer, BertTokenizerFast
import nltk
import spacy
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

nltk.download('punkt')
nltk.download('punkt_tab')
nlp = spacy.load("en_core_web_sm")

# from sklearn.metrics.pairwise import cosine_similarity
# from sentence_transformers import SentenceTransformer

# Standard library imports
import os
import re
from collections import Counter, defaultdict
from itertools import combinations

# Third-party imports
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import spacy

from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration')


# nltk downloads and imports
from nltk.corpus import stopwords
from nltk.tokenize import (
    RegexpTokenizer,
    TextTilingTokenizer,
    sent_tokenize,
    word_tokenize,
)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import json

# connor_dataset_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/Reprocessed_Connor_qa_pairs.json'
# befikir_dataset_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/qa_befikir.json'
# cameron_dataset_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/Manual_QA_Dataset_v2.json'

# with open(connor_dataset_path, 'r') as f:
#     connor_manual_qa_pairs = json.load(f)

# with open(befikir_dataset_path, 'r') as f:
#     befikir_manual_qa_pairs = json.load(f)

# with open(cameron_dataset_path, 'r') as f:
#     cameron_manual_qa_pairs = json.load(f)

# def update_positions(dataset):
#     for qa in dataset:
#         context = qa['context']
#         answer = qa['answer']

#         start_position = context.find(answer)
#         end_position = start_position + len(answer) if start_position != -1 else -1

#         qa['start_position'] = start_position
#         qa['end_position'] = end_position

# update_positions(connor_manual_qa_pairs)
# update_positions(befikir_manual_qa_pairs)
# update_positions(cameron_manual_qa_pairs)

# combined_qa_pairs = []

# combined_qa_pairs.extend(connor_manual_qa_pairs)
# combined_qa_pairs.extend(befikir_manual_qa_pairs)
# combined_qa_pairs.extend(cameron_manual_qa_pairs)

# output_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/Final_Manual_QA_Dataset.json'
# with open(output_path, 'w') as f:
#     json.dump(combined_qa_pairs, f, indent=4)

# print(f"Combined dataset saved to {output_path}")


Combined dataset saved to /content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/Final_Manual_QA_Dataset.json


In [ ]:
contractions_dict = {
      "ain’t": "are not", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have",
      "'cause": "because", "could’ve": "could have", "couldn’t": "could not", "couldn’t’ve": "could not have",
      "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not",
      "hadn’t’ve": "had not have", "hasn’t": "has not", "haven’t": "have not", "he’d": "he would",
      "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "how’d": "how did",
      "how’d’y": "how do you", "how’ll": "how will", "I’d": "I would", "I’d’ve": "I would have",
      "I’ll": "I will", "I’ll’ve": "I will have", "I’m": "I am", "I’ve": "I have", "isn’t": "is not",
      "it’d": "it would", "it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have",
      "let’s": "let us", "ma’am": "madam", "mayn’t": "may not", "might’ve": "might have",
      "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
      "mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock",
      "oughtn’t": "ought not", "oughtn’t’ve": "ought not have", "shan’t": "shall not", "sha’n’t": "shall not",
      "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will",
      "she’ll’ve": "she will have", "should’ve": "should have", "shouldn’t": "should not",
      "shouldn’t’ve": "should not have", "so’ve": "so have", "that’d": "that would", "that’d’ve": "that would have",
      "there’d": "there would", "there’d’ve": "there would have", "they’d": "they would",
      "they’d’ve": "they would have", "they’ll": "they will", "they’ll’ve": "they will have", "they’re": "they are",
      "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would",
      "we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are",
      "we’ve": "we have", "weren’t": "were not", "what’ll": "what will", "what’ll’ve": "what will have",
      "what’re": "what are", "what’ve": "what have", "when’ve": "when have", "where’d": "where did",
      "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have", "who’ve": "who have",
      "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have",
      "would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all",
      "y’all’d": "you all would", "y’all’d’ve": "you all would have", "y’all’re": "you all are",
      "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will",
      "you’ll’ve": "you will have", "you’re": "you are", "you’ve": "you have"
  }

In [ ]:
def load_text(file_path):
    """
    Load text from a file.
    """
    print("Loading the text from the file...")
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [ ]:
def remove_punctuation(text):
    """
    Remove punctuation from a given text string.

    This function takes a string and removes any character that is not a
    word character (alphanumeric) or whitespace, effectively stripping out
    punctuation.

    Parameters:
        text (str): The input string from which to remove punctuation.

    Returns:
        str: The input string with punctuation removed.
    """
    return re.sub(r'[^\w\s]', '', text)

In [ ]:
def expand_contractions(s, contractions_dict, cont_reg):
    """
    Expand contractions in a given text string using a dictionary of contractions.

    This function searches for contractions in the input string `s` using a regular
    expression pattern (`cont_reg`) and replaces each contraction with its expanded
    form from the `contractions_dict`.

    Parameters:
        s (str): The input string containing contractions to expand.
        contractions_dict (dict): A dictionary where keys are contractions (e.g., "can't")
                              and values are their expanded forms (e.g., "cannot").
        cont_reg (re.Pattern): A regular expression pattern to identify
                           contractions within the text.

    Returns:
        str: The input string with contractions expanded.

    """
    def replace(match):
        contraction = match.group(0).lower()
        if contraction in contractions_dict:
            return contractions_dict[contraction]
        return contraction  # Returns original if not in dictionary

    return cont_reg.sub(replace, s)

In [ ]:
def remove_gutenberg_header_footer(text, start_marker, end_marker):
    """
    Remove the header and footer from Project Gutenberg text.

    Finds specified start and end markers and returns the content in between.

    Parameters:
        text (str): The text to clean.
        start_marker (str): Marker indicating the start of main content.
        end_marker (str): Marker indicating the end of main content.

    Returns:
        str: Text with header and footer removed, or original text if markers are not found.
    """
    start_index = text.find(start_marker)
    if start_index == -1:
        return text

    end_index = text.find(end_marker)
    if end_index == -1:
        return text

    cleaned_text = text[start_index + len(start_marker):end_index].strip()
    return cleaned_text


In [ ]:
def identify_chapters(text):
    """
    Identify chapter markers in text and filter.

    Finds chapter markers (e.g., "Chapter 1", "Chapter II") and returns their positions.
    Filters out chapters if they are located too closely to one another. (Table of contents)

    Parameters:
        text (str): The text in which to identify chapter markers.

    Returns:
        list of tuples: List of chapter markers and their positions, filtered by distance.
    """
    chapter_markers = []
    roman_numeral_pattern = r"(?i)\bchapter\s+(\d+|[IVXLCDM]+)\b"

    for match in re.finditer(roman_numeral_pattern, text):
        chapter_markers.append((match.group(0), match.start()))

    # Remove chapters with start index difference less than 1000
    filtered_chapter_markers = []
    for i in range(len(chapter_markers)):
        if i + 1 < len(chapter_markers):  # Check if there's a next chapter
            current_start_index = chapter_markers[i][1]
            next_start_index = chapter_markers[i + 1][1]
            if next_start_index - current_start_index >= 1000:
                filtered_chapter_markers.append(chapter_markers[i])
        else:
            # Always include the last chapter
            filtered_chapter_markers.append(chapter_markers[i])

    return filtered_chapter_markers


In [ ]:
def split_novel_into_chapters(text, chapter_markers):
    """
    Split text into chapters based on chapter marker positions.

    Parameters:
        text (str): The novel text to split.
        chapter_markers (list of tuples): Chapter markers with positions in the text.

    Returns:
        list of str: List of chapter texts.
    """
    chapters = []

    for i in range(len(chapter_markers)):
        chapter_num, chapter_start_index = chapter_markers[i]

        if i + 1 < len(chapter_markers):
            next_chapter_start_index = chapter_markers[i + 1][1]
            chapter_text = text[chapter_start_index:next_chapter_start_index].strip()
        else:
            chapter_text = text[chapter_start_index:].strip()

        if chapter_text:
            chapters.append(chapter_text)

    return chapters


In [ ]:
def combine_chapters_remove_chapter_numbers(chapters):
    """
    Combine chapter texts into one and remove chapter numbers.

    Parameters:
        chapters (list of str): List of chapter texts.

    Returns:
        str: Combined text with chapter numbers removed.
    """
    combined_text = ""
    for chapter in chapters:
        chapter = re.sub(r"Chapter\s+([IVXLC]+|\d+)\b\.?\s?.*?\n", "", chapter, flags=re.IGNORECASE)
        combined_text += chapter + "\n"  # Add a newline between chapters

    return combined_text

In [ ]:
def extract_text(file_name, start_marker, end_marker, contractions_dict):
  # Load the text from the file
  novel_text = load_text(file_name)

  # Remove header/footer and split into chapters
  novel_text_no_hf = remove_gutenberg_header_footer(novel_text, start_marker, end_marker)
  chapter_markers = identify_chapters(novel_text_no_hf)
  chapters = split_novel_into_chapters(novel_text_no_hf, chapter_markers)

  # Expand contractions in chapters
  contractions_dict = {k.lower(): v for k, v in contractions_dict.items()}
  contraction_pattern = r"\b(%s)\b" % '|'.join(map(re.escape, contractions_dict.keys()))
  cont_reg = re.compile(contraction_pattern, re.IGNORECASE)

  chapters = [expand_contractions(chapter, contractions_dict, cont_reg) for chapter in chapters]

  # Combine chapters into single text for analysis
  single_text = combine_chapters_remove_chapter_numbers(chapters)

  # tokenized_sentences = [sent_tokenize(chapter) for chapter in chapters]

  return chapters, single_text

In [ ]:

def normalize_sentence(sentence: str) -> str:

    sentence = re.sub(r'[^\w\s.!?]', '', sentence).strip()

    return sentence

def tokenize_and_normalize(text: str):
    # Tokenize the text into sentences using NLTK
    sentences = nltk.sent_tokenize(text)

    # Normalize each sentence
    normalized_sentences = [normalize_sentence(sentence) for sentence in sentences]

    return normalized_sentences

In [ ]:
file_name = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/data/a study in scarlet.txt'

start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK A STUDY IN SCARLET ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK A STUDY IN SCARLET ***"

chapters, single_text = extract_text(file_name, start_marker, end_marker, contractions_dict)
# print(chapters)
# print(single_text)

# Sentence Tokenization
normalized_sentences = tokenize_and_normalize(single_text)

# # Print normalized sentences
# for sentence in normalized_sentences:
#     print(sentence)

# # Word Tokenization
# words = []
# for sentence in normalized_sentences:
#     words.extend(word_tokenize(sentence))

# for word in words:
#     print(word)

Loading the text from the file...


In [ ]:
from transformers import pipeline, AutoTokenizer
import torch

# Detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the models
from transformers import pipeline

summary_pipeline = pipeline("summarization", model="google/pegasus-large", device=device)
qg_pipeline = pipeline("text2text-generation", model="mrm8488/t5-base-finetuned-question-generation-ap", device=device)
qa_pipeline = pipeline("question-answering", model="deepset/roberta-large-squad2", device=device)

# summary_pipeline = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
# qg_pipeline = pipeline("text2text-generation", model="valhalla/t5-base-qa-qg-hl", device=device)
# qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad", device=device)

# # Example input corpus
# normalized_sentences = [
#     "Sherlock Holmes is a fictional detective created by British author Arthur Conan Doyle.",
#     "He is known for his keen powers of observation and logical reasoning.",
#     "Holmes is often assisted by his friend and biographer, Dr. John Watson.",
#     "Together, they solve many cases that require deductive skills and intelligence.",
# ]

all_qa_pairs = []
chunk_size = 5
stride = 2

for i in range(0, len(normalized_sentences), chunk_size - stride):
    # Extract a chunk of sentences
    chunk = normalized_sentences[i:i + chunk_size]
    chunk_text = " ".join(chunk)

    # Generate summary for the chunk
    summary = summary_pipeline(chunk_text, max_length=200, min_length=20, do_sample=False)[0]["summary_text"]

    # Generate questions using the summary as context
    questions = qg_pipeline(f"generate questions: {summary}", max_new_tokens=128)

    # Generate answers using the summary as context
    for q in questions:
        question = q['generated_text']
        answer = qa_pipeline(question=question, context=summary)["answer"]

        start_position = summary.find(answer)
        end_position = start_position + len(answer)

        # Only add the QA pair if the answer is found in the context
        if start_position != -1:  # Ensure the answer exists in the context
            all_qa_pairs.append({
                "context": summary,
                "question": question,
                "answer": answer,
                "start_position": start_position,
                "end_position": end_position
            })

# # Save the QA pairs to a JSON file
import json

# Save your JSON dataset
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/qa_pairs_for_finetuning.json'

# Assuming your dataset is already in the proper format, you can save it directly:
with open(dataset_path, 'w') as f:
    json.dump(all_qa_pairs, f)
print("Dataset saved successfully!")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is

Dataset saved successfully!


In [ ]:
!pip install datasets
import json
from datasets import Dataset, load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

dataset_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/qa_pairs_for_finetuning_v2.json'

with open(dataset_path, 'r') as f:
    all_qa_pairs = json.load(f)
print("Dataset loaded successfully!")

# print(all_qa_pairs)

# Split the data into train and validation sets using `train_test_split`
train_data, val_data = train_test_split(all_qa_pairs, test_size=0.1, random_state=42)

print(train_data)
print(val_data)

# Save the new split datasets back to JSON if needed
train_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/qa_train.json'
val_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/qa_validation.json'

with open(train_path, 'w') as f:
    json.dump(train_data, f)

with open(val_path, 'w') as f:
    json.dump(val_data, f)

print("Train and validation datasets saved successfully!")

df_train = pd.DataFrame(train_data)
hf_train_dataset = Dataset.from_pandas(df_train)

print(hf_train_dataset)

df_val = pd.DataFrame(val_data)
hf_val_dataset = Dataset.from_pandas(df_val)

print(hf_val_dataset)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Dataset loaded successfully!
[{'context': 'From the Sierra Nevada to Nebraska and from the Yellowstone River in the north to the Colorado upon the south is a re

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def prepare_train_features(examples):

    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # truncate context, not the question
        max_length=512,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):

        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answer = examples["answer"][sample_index]
        answer_start_position = examples["start_position"][sample_index]
        answer_end_position = examples["end_position"][sample_index]

        # If no answers are given, set the cls_index as answer.
        if answer_start_position == -1:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answer_start_position
            end_char = answer_end_position

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

# Tokenize the dataset
tokenized_train_dataset = hf_train_dataset.map(prepare_train_features, batched=True)
tokenized_val_dataset = hf_val_dataset.map(prepare_train_features, batched=True)

Map:   0%|          | 0/647 [00:00<?, ? examples/s]

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

In [ ]:
# from transformers import BertForQuestionAnswering, Trainer, TrainingArguments

# Load the model
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results_automated",
    eval_strategy="steps",
    eval_steps=100,
    logging_dir="./logs_automated",
    logging_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    save_steps=500,
    save_total_limit=5,
    learning_rate=2e-5,
    weight_decay=0.01
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,  # Use separate dataset for evaluation in practice
)

# Fine-tune the model
trainer.train()
trainer.save_model()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
100,3.334200,2.685889
200,1.914400,2.478752
300,1.163000,2.727933
400,0.663900,3.008214
500,0.308000,3.518742
600,0.214200,3.814398
700,0.125000,4.024615
800,0.090800,4.044500


In [ ]:
drive_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration'
fine_tuned_model_save_path = os.path.join(drive_path, 'fine_tuned_model_automated')
model.save_pretrained(fine_tuned_model_save_path)
tokenizer.save_pretrained(fine_tuned_model_save_path)
print("Fine-tuned model saved successfully!")

Fine-tuned model saved successfully!


In [ ]:
manual_dataset_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/Final_Manual_QA_Dataset_v2.json'

with open(manual_dataset_path, 'r') as f:
    all_manual_qa_pairs = json.load(f)
print("Dataset loaded successfully!")

manual_train_data, manual_val_data = train_test_split(all_manual_qa_pairs, test_size=0.1, random_state=42)

manual_train_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/Manual_QA_Dataset_Train.json'
manual_val_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration/Manual_QA_Dataset_Val.json'

with open(manual_train_path, 'w') as f:
    json.dump(manual_train_data, f)

with open(manual_val_path, 'w') as f:
    json.dump(manual_val_data, f)

print("Train and validation datasets saved successfully!")

df_train_manual = pd.DataFrame(manual_train_data)
hf_train_manual_dataset = Dataset.from_pandas(df_train_manual)
print("First training sample:")
print(hf_train_manual_dataset[0])  # Debug first training sample
print("Training dataset structure:")
print(hf_train_manual_dataset)

df_val_manual = pd.DataFrame(manual_val_data)
hf_val_manual_dataset = Dataset.from_pandas(df_val_manual)

print("First validation sample:")
print(hf_val_manual_dataset[0])  # Debug first validation sample
print("Validation dataset structure:")
print(hf_val_manual_dataset)

# Tokenize the dataset
tokenized_dataset_train_manual = hf_train_manual_dataset.map(prepare_train_features, batched=True)
tokenized_dataset_val_manual = hf_val_manual_dataset.map(prepare_train_features, batched=True)

Dataset loaded successfully!
Train and validation datasets saved successfully!
First training sample:
{'context': "Holmes's sharp observation revealed a scratched key in the victim's pocket.", 'question': "What object in the victim's pocket did Holmes observe?", 'answer': 'a scratched key', 'start_position': 36, 'end_position': 51}
Training dataset structure:
Dataset({
    features: ['context', 'question', 'answer', 'start_position', 'end_position'],
    num_rows: 246
})
First validation sample:
{'context': 'Holmes had found a re-agent which was precipitated by hÃ¦moglobin and nothing else.', 'question': 'What was the re-agent Holmes discovered precipitated by?', 'answer': 'hÃ¦moglobin', 'start_position': 54, 'end_position': 65}
Validation dataset structure:
Dataset({
    features: ['context', 'question', 'answer', 'start_position', 'end_position'],
    num_rows: 28
})


Map:   0%|          | 0/246 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

In [ ]:
# Load the model
model_manual = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Define training arguments
training_args_manual = TrainingArguments(
    output_dir="./results_manual",
    eval_strategy="epoch",
    eval_steps=500,
    logging_dir="./logs_manual",
    logging_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=12,
    save_steps=1000,
    save_total_limit=2,
    learning_rate=2e-5,
)

# Initialize Trainer
trainer_manual = Trainer(
    model=model_manual,
    args=training_args_manual,
    train_dataset=tokenized_dataset_train_manual,
    eval_dataset=tokenized_dataset_val_manual,  # Use separate dataset for evaluation in practice
)

# Fine-tune the model
trainer_manual.train()


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,2.975506
2,No log,2.121408
3,No log,1.783763
4,2.618100,1.821213
5,2.618100,1.922518
6,2.618100,2.186576
7,0.608600,2.509821
8,0.608600,2.752319
9,0.608600,2.799016
10,0.171600,2.822744


TrainOutput(global_step=372, training_loss=0.9310292390085035, metrics={'train_runtime': 70.6917, 'train_samples_per_second': 41.759, 'train_steps_per_second': 5.262, 'total_flos': 771348025884672.0, 'train_loss': 0.9310292390085035, 'epoch': 12.0})

In [ ]:
drive_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration'
fine_tuned_model_save_path = os.path.join(drive_path, 'fine_tuned_model_manual')
model_manual.save_pretrained(fine_tuned_model_save_path)
tokenizer.save_pretrained(fine_tuned_model_save_path)
print("Fine-tuned model saved successfully!")

Fine-tuned model saved successfully!


In [ ]:
!pip install evaluate
!pip install bert-score
from evaluate import load
import evaluate
from bert_score import score
import torch

# Define paths
drive_path = '/content/drive/MyDrive/Colab Notebooks/Cosc524 - Collaboration'
fine_tuned_model_manual_save_path = os.path.join(drive_path, 'fine_tuned_model_manual')
fine_tuned_model_automated_save_path = os.path.join(drive_path, 'fine_tuned_model_automated')

val_path = os.path.join(drive_path, 'Manual_QA_Dataset_v1.json')

# Load pre-trained model and tokenizer directly from Hugging Face
pretrained_model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")
pretrained_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Load the fine-tuned model and tokenizer from the saved path
fine_tuned_model_manual = AutoModelForQuestionAnswering.from_pretrained(fine_tuned_model_manual_save_path)
fine_tuned_tokenizer_manual = AutoTokenizer.from_pretrained(fine_tuned_model_manual_save_path)

fine_tuned_model_automated = AutoModelForQuestionAnswering.from_pretrained(fine_tuned_model_automated_save_path)
fine_tuned_tokenizer_automated = AutoTokenizer.from_pretrained(fine_tuned_model_automated_save_path)

print("Pre-trained and fine-tuned models loaded successfully.")

# Initialize QA pipelines for both models
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available

pretrained_qa_pipeline = pipeline(
    "question-answering",
    model=pretrained_model,
    tokenizer=pretrained_tokenizer,
    device=device
)

fine_tuned_qa_manual_pipeline = pipeline(
    "question-answering",
    model=fine_tuned_model_manual,
    tokenizer=fine_tuned_tokenizer_manual,
    device=device
)

fine_tuned_qa_automated_pipeline = pipeline(
    "question-answering",
    model=fine_tuned_model_automated,
    tokenizer=fine_tuned_tokenizer_automated,
    device=device
)

print("QA Pipelines initialized successfully.")

# Load the validation dataset
with open(val_path, 'r') as f:
    val_data = json.load(f)
print("Validation dataset loaded successfully!")

# Prepare lists to store predictions
pretrained_predictions = []
fine_tuned_manual_predictions = []
fine_tuned_automated_predictions = []

# Generate predictions with the pre-trained model
print("Generating predictions with the pre-trained model...")
for i, qa in enumerate(val_data):
    result = pretrained_qa_pipeline(question=qa["question"], context=qa["context"])
    pretrained_predictions.append({
        "id": str(i),
        "prediction_text": result['answer']
    })

    # Print question, predicted answer, and expected answer for verification
    print(f"Pre-trained Model - Question: {qa['question']}")
    print(f"Predicted Answer: {result['answer']}")
    print(f"Expected Answer: {qa['answer']}\n")

# Generate predictions with the fine-tuned model - QA generated maually
print("Generating predictions with the fine-tuned model - QA generated manually...")
for i, qa in enumerate(val_data):
    result = fine_tuned_qa_manual_pipeline(question=qa["question"], context=qa["context"])
    fine_tuned_manual_predictions.append({
        "id": str(i),
        "prediction_text": result['answer']
    })

    # Print question, predicted answer, and expected answer for verification
    print(f"Fine-tuned Model Manual - Question: {qa['question']}")
    print(f"Predicted Answer: {result['answer']}")
    print(f"Expected Answer: {qa['answer']}\n")

# Generate predictions with the fine-tuned model - QA generated automatically
print("Generating predictions with the fine-tuned model - QA generated automatically...")
for i, qa in enumerate(val_data):
    result = fine_tuned_qa_automated_pipeline(question=qa["question"], context=qa["context"])
    fine_tuned_automated_predictions.append({
        "id": str(i),
        "prediction_text": result['answer']
    })

    # Print question, predicted answer, and expected answer for verification
    print(f"Fine-tuned Model Automated - Question: {qa['question']}")
    print(f"Predicted Answer: {result['answer']}")
    print(f"Expected Answer: {qa['answer']}\n")

# Prepare references in SQuAD format
references = [{
    "id": str(i),
    "answers": {
        "text": [qa["answer"]],
        "answer_start": [qa["start_position"]]
    }
} for i, qa in enumerate(val_data)]

# Load the SQuAD metric
squad_metric = evaluate.load("squad")

# Compute metrics for the pre-trained model
print("Computing SQuAD metrics for the pre-trained model...")
pretrained_metrics = squad_metric.compute(predictions=pretrained_predictions, references=references)
print("Pre-trained BERT SQuAD Metrics:")
print(pretrained_metrics)

# Compute metrics for the fine-tuned model - QA manual
print("Computing SQuAD metrics for the fine-tuned model - QA manual...")
fine_tuned_metrics_manual = squad_metric.compute(predictions=fine_tuned_manual_predictions, references=references)
print("Fine-tuned BERT SQuAD Metrics - QA manual:")
print(fine_tuned_metrics_manual)

# Compute metrics for the fine-tuned model - QA automated
print("Computing SQuAD metrics for the fine-tuned model - QA automated...")
fine_tuned_metrics_automated = squad_metric.compute(predictions=fine_tuned_automated_predictions, references=references)
print("Fine-tuned BERT SQuAD Metrics - QA manual:")
print(fine_tuned_metrics_automated)


# Prepare predictions as lists of strings for BERTScore
pretrained_predictions_text = [d['prediction_text'] for d in pretrained_predictions]
fine_tuned_manual_predictions_text = [d['prediction_text'] for d in fine_tuned_manual_predictions]
fine_tuned_automated_predictions_text = [d['prediction_text'] for d in fine_tuned_automated_predictions]

# Prepare references as lists of strings for BERTScore
references_text = [d['answers']['text'][0] for d in references]

# Compute BERTScore
print("Computing BERTScore for the pre-trained model...")
P_PT, R_PT, F1_PT = score(pretrained_predictions_text, references_text, lang="en", model_type="bert-base-uncased")

# Print scores
print(f"Precision: {P_PT.mean():.4f}")
print(f"Recall: {R_PT.mean():.4f}")
print(f"F1 Score: {F1_PT.mean():.4f}")

# Compute BERTScore
print("Computing BERTScore for the fine-tuned model - QA manual...")
P_FTM, R_FTM, F1_FTM = score(fine_tuned_manual_predictions_text, references_text, lang="en", model_type="bert-base-uncased")

# Print scores
print(f"Precision: {P_FTM.mean():.4f}")
print(f"Recall: {R_FTM.mean():.4f}")
print(f"F1 Score: {F1_FTM.mean():.4f}")

# Compute BERTScore
print("Computing BERTScore for the fine-tuned model - QA automated...")
P_FTA, R_FTA, F1_FTA = score(fine_tuned_automated_predictions_text, references_text, lang="en", model_type="bert-base-uncased")

# Print scores
print(f"Precision: {P_FTA.mean():.4f}")
print(f"Recall: {R_FTA.mean():.4f}")
print(f"F1 Score: {F1_FTA.mean():.4f}")

# Compare Exact Match (EM) and F1 scores
comparison_detailed = pd.DataFrame({
    "Metric": ["exact_match", "f1"],
    "Pre-trained BERT": [pretrained_metrics.get("exact_match", 0), pretrained_metrics.get("f1", 0)],
    "Fine-tuned BERT - QA Manual": [fine_tuned_metrics_manual.get("exact_match", 0), fine_tuned_metrics_manual.get("f1", 0)],
    "Fine-tuned BERT - QA Automated": [fine_tuned_metrics_automated.get("exact_match", 0), fine_tuned_metrics_automated.get("f1", 0)]
})

print("\nDetailed Comparison of Pre-trained and Fine-tuned BERT Models:")
print(comparison_detailed)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Pre-trained and fine-tuned models loaded successfully.
QA Pipelines initialized successfully.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Validation dataset loaded successfully!
Generating predictions with the pre-trained model...
Pre-trained Model - Question: Who introduced Watson to Holmes?
Predicted Answer: recently returned from
Expected Answer: Stamford

Pre-trained Model - Question: Where did Watson and Holmes live?
Predicted Answer: recently returned from
Expected Answer: 221B Baker Street

Pre-trained Model - Question: What word was written in blood at the crime scene?
Predicted Answer: recently returned from Afghanistan, is introduced to Sherlock Holmes
Expected Answer: RACHE

Pre-trained Model - Question: Who was revealed as the murderer?
Predicted Answer: recently returned from Afghanistan, is introduced to Sherlock Holmes
Expected Answer: Jefferson Hope

Pre-trained Model - Question: How did Holmes deduce that Watson had served in Afghanistan?
Predicted Answer: Holmes also has an eccentric personality, often
Expected Answer: by observing his appearance and behavior

Pre-trained Model - Question: What instrume

Computing SQuAD metrics for the pre-trained model...
Pre-trained BERT SQuAD Metrics:
{'exact_match': 0.0, 'f1': 8.424369747899158}
Computing SQuAD metrics for the fine-tuned model - QA manual...
Fine-tuned BERT SQuAD Metrics - QA manual:
{'exact_match': 17.647058823529413, 'f1': 32.54421418086961}
Computing SQuAD metrics for the fine-tuned model - QA automated...
Fine-tuned BERT SQuAD Metrics - QA manual:
{'exact_match': 19.11764705882353, 'f1': 34.51607612481533}
Computing BERTScore for the pre-trained model...
Precision: 0.4386
Recall: 0.3873
F1 Score: 0.4002
Computing BERTScore for the fine-tuned model - QA manual...
Precision: 0.5192
Recall: 0.5692
F1 Score: 0.5337
Computing BERTScore for the fine-tuned model - QA automated...
Precision: 0.5594
Recall: 0.5869
F1 Score: 0.5651

Detailed Comparison of Pre-trained and Fine-tuned BERT Models:
        Metric  Pre-trained BERT  Fine-tuned BERT - QA Manual  \
0  exact_match           0.00000                    17.647059   
1           f1 